In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install Pillow numpy glob2 regex os-sys argparse matplotlib 
!pip install med2image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import time
import sys

In [ ]:
def sort_list(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [ ]:
def create_image(dirname, dirname2, label, filename):
    if dirname[-1] == "/":
        dirname = dirname[:-1]
    images = glob.glob(str(dirname) + "/*.png")

    if dirname2[-1] == "/":
        dirname2 = dirname2[:-1]
    images2 = glob.glob(str(dirname2) + "/*.png")

    # Sort images alphanumerically by filename to ensure that z loops from front to back
    images = sort_list(images)
    images2 = sort_list(images2)

    p = np.zeros((len(images), Image.open(images[0]).convert('L').size[0]))

    # setup toolbar
    toolbar_width = int(len(images)/5 + 1)
    sys.stdout.write(label + " [%s]" % (" " * toolbar_width))
    sys.stdout.flush()
    sys.stdout.write("\b" * (toolbar_width+1))

    # Loop through CT slices from front to back
    for z in range(len(images)):
        img = Image.open(images[z]).convert('L')  # convert image to 8-bit grayscale
        HEIGHT, WIDTH = img.size
        data = list(img.getdata()) # convert image data to a list of integers
        # convert that to 2D list (list of lists of integers)
        pixels = [data[offset:offset+WIDTH] for offset in range(0, WIDTH*HEIGHT, WIDTH)]

        img2 = Image.open(images2[z]).convert('L') 
        data2 = list(img2.getdata()) 
        pixels2 = [data2[offset:offset+WIDTH] for offset in range(0, WIDTH*HEIGHT, WIDTH)]

        for r in range(HEIGHT):
            for c in range(WIDTH):
                if pixels2[r][c] != 0:
                    pixels[r][c] = pixels2[r][c]

        # Loop from left to right on the CT slice
        for x in range(WIDTH):
            # Sum y values in the current x column
            sum = 0
            for y in range(HEIGHT):
                sum += pixels[y][x]
            # Assign sum to the point (x, z) on the coronal image - p[z][x] in the pixel array, 
            # since z represents height (rows) and x represents length (columns)
            p[len(images) - 1 - z][x] = sum

        if z % 5 == 0:
            # update the bar
            sys.stdout.write("-")
            sys.stdout.flush()
    sys.stdout.write("]\n")
    
    plt.clf()
    plt.imshow(p, cmap='gray')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.savefig(filename, bbox_inches = 'tight',
        pad_inches = 0)
    
    final_img = Image.open(filename)
    size = 300
    if final_img.size[1] < 300:
        final_img = final_img.resize((final_img.size[0], 300))    

    final_img.save(filename)

In [ ]:
scans = glob.glob('/content/drive/My Drive/data/CT_Scans/*.nii')
scans = sort_list(scans)
for i in range(len(scans)):
  name = './data/CT_Scans/Volume' + str(i) + '/'
  tf.io.gfile.makedirs(name)
  scan = scans[i]
  scan = './' + scan[scan.rindex('data'):]
  !med2image -i $scan -o $name

masks = glob.glob('/content/drive/My Drive/data/Infection_Masks/*.nii')
masks = sort_list(masks)
for i in range(len(masks)):
  name = './data/Infection_Masks/Volume' + str(i) + '/'
  tf.io.gfile.makedirs(name)
  scan = masks[i]
  scan = './' + scan[scan.rindex('data'):]
  !med2image -i $scan -o $name

In [ ]:
from google.colab import files
import tensorflow as tf

%cd /content/drive/My Drive/

for i in range(1):
  scan_name = './data/CT_Scans/'
  mask_name = './data/Infection_Masks/Volume' + str(i)
  create_image(scan_name, mask_name, "Creating X-ray:", 'xray' + str(i) + '.png')
  files.download('xray' + str(i) + '.png')

for i in range(19):
  scan_name = './data/CT_Scans/Volume' + str(i+1)
  mask_name = './data/Infection_Masks/Volume' + str(i+1)
  create_image(scan_name, mask_name, "Creating X-ray:", 'xray' + str(i+1) + '.png')
  files.download('xray' + str(i+1) + '.png')    